In [11]:
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import time
from bs4 import BeautifulSoup
import re
import random

In [12]:
target_url = "https://www.sahibinden.com/satilik/aydin-kusadasi-davutlar?sorting=price_asc&query_text_mf=davutlar"
number_of_pages = 50 # how many pages to scrape

In [13]:
# start web browser
browser = webdriver.Chrome('/home/ahmetd/Downloads/chromedriver')
options = webdriver.ChromeOptions();
options.add_argument('headless');
options.add_argument('window-size=1200x600'); ## optional

In [14]:
def process_html(html_in):
    soup = BeautifulSoup(html_in, "html.parser")
    results = soup.findAll( "tr", {"class" : "searchResultsItem"} )
                
    page_entries = set()
    for i, val1 in enumerate(results):
        my_list = []
        id_no = val1.get('data-id')
        if (id_no == "") or (id_no == None): continue
        my_list.append(id_no)
        #print(key)
        for j, val2 in enumerate(val1):
            #print (val2)
            
            if len(val2) > 1:
                    #print(60*"oO")
                        #print (val2.text)
                new_val = re.sub('\\n\\n', " ", val2.text)
                my_list.append(new_val.strip())
            elif (val2 != "\n") and (val2 != ""):
                val2 = str(val2)
                new_val = re.sub('<td class=\"search\S+\">', "", val2)
                new_val2 = re.sub('</td>', "", new_val)
                my_list.append(new_val2.strip())
                
        page_entries.add(tuple(my_list))
    #print("page_entries:  {}".format(page_entries))
        
    return page_entries


In [15]:
current_page = 0
browser.get(target_url)
all_entries = set()
while current_page < number_of_pages:
    
    current_page+=1
    # get sourcecode
    print( "Current page is: {} / {}".format(current_page, number_of_pages) )
    html_received = browser.page_source
    entries_in_page = process_html(html_received)
    #print("entries_in_page: {}".format(entries_in_page))
    all_entries = all_entries.union(entries_in_page) #returns set
    #print("all_entries: {}".format(all_entries))
    time.sleep(random.uniform(15,20))
    try:
        button = browser.find_element_by_xpath("//a[contains(.,'Sonraki')]")
        browser.execute_script("arguments[0].click();", button)


    except NoSuchElementException:
        print("Scraping terminated before reaching target number of pages. Needed {}, got {}.".format(number_of_pages, current_page))
    except ElementClickInterceptedException:
        print("NOOOOOOOOOOOOOOOOOO! We have been found!!ABORT ABORT!!")

#print("all_entries: {}".format(all_entries))

Current page is: 1 / 50
Current page is: 2 / 50
Current page is: 3 / 50
Current page is: 4 / 50
Current page is: 5 / 50
Current page is: 6 / 50
Current page is: 7 / 50
Current page is: 8 / 50
Current page is: 9 / 50
Current page is: 10 / 50
Current page is: 11 / 50
Current page is: 12 / 50
Current page is: 13 / 50
Current page is: 14 / 50
Current page is: 15 / 50
Current page is: 16 / 50
Current page is: 17 / 50
Current page is: 18 / 50
Current page is: 19 / 50
Current page is: 20 / 50
Current page is: 21 / 50
Current page is: 22 / 50
Current page is: 23 / 50
Current page is: 24 / 50
Current page is: 25 / 50
Current page is: 26 / 50
Current page is: 27 / 50
Current page is: 28 / 50
Current page is: 29 / 50
Current page is: 30 / 50
Current page is: 31 / 50
Current page is: 32 / 50
Current page is: 33 / 50
Current page is: 34 / 50
Current page is: 35 / 50
Current page is: 36 / 50
Current page is: 37 / 50
Current page is: 38 / 50
Current page is: 39 / 50
Current page is: 40 / 50
Current p

In [16]:
#close web browser
browser.close()

In [17]:
all_entries = list(all_entries)
import pandas as pd
df = pd.DataFrame(all_entries, 
                  columns=['ID', 'dum1', 'House_Type', 'Message', 'Area', 
                           'Rooms', 'Price', 'Date','Location','Dum2'])


In [18]:
df = df.drop('dum1', axis=1)
df = df.drop('Dum2', axis=1)
df.head()

,ID,House_Type,Message,Area,Rooms,Price,Date,Location
0,832559570,Daire,DAVUTLARDA 550 BİNDEN BAŞLAYAN FİYATLARA 2+1 H...,170,2+1,550.000 TL,16 Temmuz 2020,Davutlar
1,844487204,Daire,Kuşadası davutlar merkezde sıfır satılık 3+1 d...,180,3+1,279.000 TL,26 Temmuz 2020,Davutlar
2,848973395,Daire,davutlar da havuzlu full eşyalı mükemmel bir 1+1,65,1+1,325.000 TL,10 Ağustos 2020,Davutlar
3,780988049,Daire,GÜZELÇAMLI ANAHTAR TESLIM ULTRA LUX AVANTAJLI ...,95,2+1,345.000 TL,05 Ağustos 2020,Güzelçamlı Mh.
4,843614799,Daire,FİLİZ EMLAKTAN EMSALSİZ BÜYÜKLÜKTE SATILIK 2+1...,110,2+1,359.000 TL,23 Temmuz 2020,Davutlar


In [19]:
df.to_csv("sahibinden.csv", index=False)

In [20]:
df.tail()

,ID,House_Type,Message,Area,Rooms,Price,Date,Location
974,832535213,Yazlık,MİS**den**HAVUZLU SİTE İÇİNDE**5+1**DENİZE YÜR...,200,5+1,600.000 TL,17 Temmuz 2020,Davutlar
975,844244127,Daire,MERKEZDE SATILIK 2+1 FULL MANZARALI FERAH DAİRE,85,2+1,265.000 TL,25 Temmuz 2020,Davutlar
976,833393178,Daire,YAZLIK + KIŞLIK,120,3+1,425.000 TL,05 Ağustos 2020,Davutlar
977,843736307,Daire,KUŞADASI DAVUTLAR MERKEZDE SIFIR 2+1 AYRI MUTF...,115,2+1,335.000 TL,23 Temmuz 2020,Davutlar
978,841117901,Villa,KUŞADASI DAVUTLAR'DA 12 YAŞINDA ÇOK FERAH HİÇ ...,225,4+1,620.000 TL,13 Ağustos 2020,Davutlar
